In [ ]:
# upload magenta-master.zip to google drive and unzip it
# !unzip -q magenta-master.zip

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
%cd gdrive/MyDrive

/content/gdrive/MyDrive


In [ ]:
# also open up this directory in google drive to get the outputs
%cd magenta-master/
%cd magenta/models/performance_rnn/

/content/gdrive/MyDrive/magenta-master
/content/gdrive/MyDrive/magenta-master/magenta/models/performance_rnn


In [ ]:
!pip install magenta

num_outputs = The number of tracks to generate. One MIDI file will be created for each

num_steps = The total number of steps the generated track should be, priming track length + generated steps. Each step is 10 milliseconds.

The current primer 'starter_15s.midi' is 15 seconds. There's another shorter and faster one 'starter_8s.midi'. You guys can find a better one that has multiple instruments. This model can handle polyphony.

After selecting the best generated music from each iteration on MTurk, use the best music from the output directory as the new primer and change the primer_time to its duration.

In [ ]:
BUNDLE_PATH='./performance_with_dynamics.mag'
CONFIG='performance_with_dynamics'

# Parameters To Change 

In [ ]:
# change the primer time and path after each iteration
primer_time = 8
PRIMER_PATH = './starter_8s.midi'
# PRIMER_PATH = './model_out/2021-04-27_062830_01.mid'
iter_num = 1

# keep these parameters fixed accross iterations
gen_new_time = 15
n_outputs = 15

# no change
num_steps = (primer_time + gen_new_time) * 100 
out_dir = './model_out/iter' + str(iter_num) + '/'
mp3_out_dir = './QC_out/mp3_out/iter' + str(iter_num) + '/'

# Generate

In [ ]:
!python performance_rnn_generate.py \
--config={CONFIG} \
--bundle_file={BUNDLE_PATH} \
--output_dir={out_dir} \
--num_outputs={n_outputs} \
--num_steps={num_steps} \
--primer_midi={PRIMER_PATH}

2021-04-27 18:45:40.146180: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
/usr/local/lib/python3.7/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/usr/local/lib/python3.7/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
Instructions for updating:
non-resource variables are not supported in the lo

# QC part 

will convert .mid files into .mp3 with QC added and store into the directory QC_out

In [ ]:
!pip install gTTS
!pip install pydub
!pip install midi2audio
!apt install fluidsynth
!cp /usr/share/sounds/sf2/FluidR3_GM.sf2 ./font.sf2

In [ ]:
from gtts import gTTS 
from random import randint
from pydub import AudioSegment
from midi2audio import FluidSynth
from IPython.display import Audio
import pandas as pd
import os
from os import listdir
from os.path import isfile, join

In [ ]:
mid_fps = [join(out_dir, f) for f in listdir(out_dir) if isfile(join(out_dir, f))]

In [ ]:
def getSpeech():
  num = randint(10, 99)
  text = "Please enter" + str(num) + "into the textbox"
  language = 'en'
  speech = gTTS(text = text, lang = language, slow = False)
  return num, speech

def combineMusicSpeech (midi_fp):
  FluidSynth("font.sf2").midi_to_audio(midi_fp, 'music.wav')
  num, speech = getSpeech()
  speech.save('text.mp3')
  sound1 = AudioSegment.from_file('music.wav', format="wav")
  sound2 = AudioSegment.from_file("text.mp3", format="mp3")
  combined = sound1 + sound2
  return num, combined

In [ ]:
fps_nums = []
for i, mid_fp in enumerate(mid_fps):
  mid_fn = mid_fp.split('.')[1].split('/')[-1]
  mp3_fn = mid_fn + ".mp3"
  num, combined = combineMusicSpeech(mid_fp)
  combined.export(mp3_out_dir + mp3_fn, format="mp3")
  # change this output filepath to some github or web url + mp3_fp,
  # and make sure to upload all files in the mp3_out_dir to that url 
  url = 'https://raw.github.com/selveszero/nets213final/main/data/mp3_files/iter' + str(iter_num) + '/'
  mp3_fp = url + mp3_fn
  fps_nums.append((mp3_fp, num))

In [ ]:
df = pd.DataFrame(fps_nums, columns = ["audio_url", "number"])
df.to_csv("./QC_out/filepaths_and_numbers_iter"+ str(iter_num) + ".csv", index=False)